In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/voicegender/voice.csv")

In [ ]:
data.head()


In [ ]:
data.label = [1 if i=='male' else 0 for i in data.label]
x = data.drop(["label"],axis=1)
y = data.label.values 

So we separated the label column. We assigned 1 for male and 0 for 
female.

## Normalization

* Let's normalize the values of x between 0 and 1

In [ ]:
x = (x-np.min(x))/(np.max(x)-np.min(x)).values


 * Let's split our data for training and testing

In [ ]:
from sklearn.model_selection import train_test_split
    
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=20)

x_train = x_train.T
x_test = x_test.T
y_train = y_train.T
y_test = y_test.T

## Sigmoid

In [ ]:
def init(dimension):
    w = np.full((dimension,1),0.01)
    b = 0.0
    return w,b

def sigmoid(z):
    y_head = 1/(1+np.exp(-z))
    return y_head

## Forward and Backward Propagation

In [ ]:
def forward_backward_pro(w,b,x_train,y_train):
    z = np.dot(w.T,x_train) + b
    y_head = sigmoid(z)
    loss = -(y_train*np.log(y_head) + (1-y_train)*np.log(1-y_head))
    cost = np.sum(loss)/(x_train.shape[1])
    
    weight = (np.dot(x_train,(y_head-y_train).T))/x_train.shape[1]
    bias = np.sum(y_head-y_train)/x_train.shape[1]
    
    grad = {"weight":weight, "bias":bias}
    
    return cost,grad
    

## Update

In [ ]:
def update(w,b,x_train,y_train,learning_rate,num):
    cost_list = []
    index = []
    
    for i in range(num):
        cost,grad = forward_backward_pro(w,b,x_train,y_train)
        
        w = w - learning_rate * grad["weight"]
        b = b - learning_rate * grad["bias"]
        if i % 100 == 0:
            cost_list.append(cost)
            index.append(i)
            print ("Cost after iteration %i: %f" %(i, cost))
        
        
    parameters = {"weight":w, "bias":b}
    plt.plot(index,cost_list)
    plt.xlabel("number of iteration")
    plt.ylabel("cost")
    plt.show()
    return parameters,grad,cost_list

## Predict

In [ ]:
def predict(w,b,x_test):
    z = sigmoid(np.dot(w.T,x_test)+b)
    prediction = np.zeros((1,x_test.shape[1]))
    
    for i in range(z.shape[1]):
        if(z[0,i]<=0.5):
            prediction[0,i]=0
        else:
            prediction[0,i]=1
    return prediction

In [ ]:
def logistic_regression(x_train,y_train,x_test,y_test,learning_rate,num):
    w,b = init(x_train.shape[0])
    parameters,grad,cost_list = update(w,b,x_train,y_train,learning_rate,num)
    
    prediction = predict(parameters["weight"],parameters["bias"],x_test)
    print("test accuracy: {} %".format(100 - np.mean(np.abs(prediction - y_test)) * 100))

In [ ]:
logistic_regression(x_train,y_train,x_test,y_test,learning_rate=0.1,num=5000)